# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,urangan,-25.3000,152.9000,294.48,70,100,7.34,AU,1728856253
1,1,kisangani,0.5167,25.2000,294.63,94,31,0.54,CD,1728856254
2,2,vorgashor,67.5833,63.9500,271.69,95,55,6.27,RU,1728856255
3,3,ushuaia,-54.8000,-68.3000,281.96,71,40,7.72,AR,1728856257
4,4,port-aux-francais,-49.3500,70.2167,277.08,90,93,10.38,TF,1728856258


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng" ,
    "Lat" ,
    geo= True , 
    tiles = "OSM" ,
    size = "Humidity" ,
    color = "City" ,
    hover_cols = ["Lng" , "Lat" , "City" , "Humidity"] ,
    xlabel = "Longitude" ,
    ylabel = "latitude" ,
    

)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[ (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,hammerfest,70.6634,23.6821,278.40,70,0,1.54,NO,1728856278
51,51,lobujya,27.9500,86.8167,264.59,58,0,2.36,NP,1728856322
93,93,great falls,47.5002,-111.3008,293.24,38,0,4.12,US,1728856200
97,97,al markaz,25.4000,49.7333,301.48,72,0,1.33,SA,1728856387
106,106,puerto deseado,-47.7503,-65.8938,286.53,48,0,3.91,AR,1728856366


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City" , "Country" , "Lat" , "Lng" , "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[: , "Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\bavth\AppData\Local\Temp\ipykernel_21864\1342524955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[: , "Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
18,hammerfest,NO,70.6634,23.6821,70,
51,lobujya,NP,27.9500,86.8167,58,
93,great falls,US,47.5002,-111.3008,38,
97,al markaz,SA,25.4000,49.7333,72,
106,puerto deseado,AR,-47.7503,-65.8938,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel" , 
    "api_key" : geoapify_key , 
    "limit" : 1 , 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params= params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
hammerfest - nearest hotel: Thon Hotel Hammerfest
lobujya - nearest hotel: Hôtel Mother Earth
great falls - nearest hotel: La Quinta Inn & Suites
al markaz - nearest hotel: No hotel found
puerto deseado - nearest hotel: Los Olmos
lamar - nearest hotel: 1st St Lofts
mammoth lakes - nearest hotel: Travelodge Mammoth Lakes
lubango - nearest hotel: Novo Hotel
yuzhno-kurilsk - nearest hotel: Айсберг
cabo san lucas - nearest hotel: Comfort Rooms
villa elisa - nearest hotel: Hotel Yatch y Golf Club
stanley - nearest hotel: Hotel 52
prince rupert - nearest hotel: Crest Hotel
agua prieta - nearest hotel: HOTEL REAL DE AGUA PRIETA
cinisi - nearest hotel: Cinisi Vacanze
inhambane - nearest hotel: Hotel de Inhambane
rawlins - nearest hotel: Quality Inn Rawlins I-80
tsiombe - nearest hotel: No hotel found
shiyan - nearest hotel: 汉庭酒店
thal - nearest hotel: No hotel found
nemuro - nearest hotel: お宿エクハシ
abashiri - nearest hotel: ホテル オホーツク・イン
saudarkrokur - nearest hotel: Mikligar

,City,Country,Lat,Lng,Humidity,Hotel Name
18,hammerfest,NO,70.6634,23.6821,70,Thon Hotel Hammerfest
51,lobujya,NP,27.9500,86.8167,58,Hôtel Mother Earth
93,great falls,US,47.5002,-111.3008,38,La Quinta Inn & Suites
97,al markaz,SA,25.4000,49.7333,72,No hotel found
106,puerto deseado,AR,-47.7503,-65.8938,48,Los Olmos
118,lamar,US,33.6668,-95.5836,36,1st St Lofts
119,mammoth lakes,US,37.6485,-118.9721,15,Travelodge Mammoth Lakes
126,lubango,AO,-14.9172,13.4925,94,Novo Hotel
152,yuzhno-kurilsk,RU,44.0306,145.8556,60,Айсберг
155,cabo san lucas,MX,22.8909,-109.9124,66,Comfort Rooms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng" , "Lat" ,
    geo = True,
    tiles = "OSM" ,
    size = "Humidity" ,
    color = "Humidity" ,
    hover_cols = ["City" , "Country" , "Humidity" ,"Hotel Name"] , 
    xlabel = "Longitude" ,
    ylabel = "latitude" ,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)